# Disaster crisis NLP Project (Nepal Earthquake dataset and Queensland_FLD datasets)

## Import packages 

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None  # default='warn' surpress chain assignment warning

from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

import nltk
nltk.download('all') 

import numpy as np

import re                                  
import string
from nltk.corpus import wordnet
from nltk.corpus import brown
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer


from textblob import Word

from collections import Counter

import gensim

from numpy import savetxt

# XGBoost
import xgboost as xgb
from xgboost import XGBClassifier

# sklearn 
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import coo_matrix, hstack

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

import torch

# turn off userwarnings
import warnings
warnings.filterwarnings('ignore')

#pip install tokenization

Mounted at /content/drive


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

## Data collection

In [ ]:
# from google.colab import files
# uploaded_Nepal_EQ = files.upload()
# uploaded_Queensland_FLD = files.upload()

In [ ]:
# import io
# Nepal_EQ = pd.read_csv(io.BytesIO(uploaded_Nepal_EQ['2015_Nepal_Earthquake_train.tsv']), delimiter='\t',  encoding='iso-8859-1')
# Queensland_FLD = pd.read_csv(io.BytesIO(uploaded_Queensland_FLD['train.csv']))
# # Dataset is now stored in a Pandas Dataframe

In [ ]:
Nepal_EQ_path = '/content/drive/MyDrive/ACL_ICWSM_2018_datasets/nepal/2015_Nepal_Earthquake_train.tsv'
Queensland_FLD_path = '/content/drive/MyDrive/ACL_ICWSM_2018_datasets/queensland/2013_Queensland_Floods_train.tsv'

# Load TSV file into a Pandas DataFrame
Nepal_EQ = pd.read_csv(Nepal_EQ_path, delimiter='\t',  encoding='iso-8859-1')

Queensland_FLD = pd.read_csv(Queensland_FLD_path, delimiter='\t',  encoding='iso-8859-1')

In [ ]:
# # Load TSV file into a Pandas DataFrame
# Nepal_EQ = pd.read_csv('/Users/musaphiri/Downloads/Disaster Crisis Dataset/ACL_ICWSM_2018_datasets/nepal/2015_Nepal_Earthquake_train.tsv', 
#                  delimiter='\t',  encoding='iso-8859-1')

# Queensland_FLD = pd.read_csv('/Users/musaphiri/Downloads/Disaster Crisis Dataset/nlp-getting-started/train.csv') 

In [ ]:
# Print the first five rows of the Nepal Earthquake DataFrame
Nepal_EQ.head()

,tweet_id,text,label
0,591902739002560512,"RT @AnupKaphle: #Nepal's prime minister addressed the country for 1st time since earthquake on Saturday. No concrete plans, lots of referenâ°Ã_",relevant
1,592939706788216832,@jonsnowC4 So have we; read our friends blog from Lamjung where they are working http://t.co/jGpSacUQpe,relevant
2,592591542168252416,Lend a helping hand if you can #Nepal https://t.co/FdRrvC84EA,relevant
3,591903009279385600,@shilpaanand they've managed to reach Kathmandu but with the help of their guide - not through the official evacuation I believe.,relevant
4,592099765271199744,Israel Sending Aid Teams to Nepal After Quake: Israel was sending a delegation to Nepal Saturday to determine ... http://t.co/fjRuwl9vZI,relevant


In [ ]:
# Print the first five rows of the Queensland floods DataFrame
Queensland_FLD.head()

,tweet_id,text,label
0,295541465013182465,I just though about the night I went clubbing with @mikal1988 and I cried,not_relevant
1,295485717465935873,Looks like its going to be another long night courtesy #djoko and #murray #AustralianOpen,not_relevant
2,297292225811972097,@LaniiBanani hahahaha I just told him id have to think about it :p he was like what I would do to u if u were in my arms + more haha,not_relevant
3,297199686392094720,Off to meeting.... with so called... Baaps of mineral processing!!,not_relevant
4,295527618449666049,Doubt I'll be getting much sleep tonight,not_relevant


In [ ]:
# Print Nepal_EQ dataset information 
Nepal_EQ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6899 entries, 0 to 6898
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet_id  6899 non-null   int64 
 1   text      6899 non-null   object
 2   label     6899 non-null   object
dtypes: int64(1), object(2)
memory usage: 161.8+ KB


In [ ]:
# Print Queensland_FLD dataset information 
Queensland_FLD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6019 entries, 0 to 6018
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet_id  6019 non-null   int64 
 1   text      6019 non-null   object
 2   label     6019 non-null   object
dtypes: int64(1), object(2)
memory usage: 141.2+ KB


## Data Preprocessing 

In [ ]:
# Create new Nepal_EQ dataset with only 2 features (text & label) 
NepalEQ_df = Nepal_EQ[['text', 'label']]
NepalEQ_df.head(3)

,text,label
0,"RT @AnupKaphle: #Nepal's prime minister addressed the country for 1st time since earthquake on Saturday. No concrete plans, lots of referenâ°Ã_",relevant
1,@jonsnowC4 So have we; read our friends blog from Lamjung where they are working http://t.co/jGpSacUQpe,relevant
2,Lend a helping hand if you can #Nepal https://t.co/FdRrvC84EA,relevant


# Create new Queensland_FLD dataset with only 2 features (text & label) 


In [ ]:
QueenslandFLD_df = Queensland_FLD[['text', 'label']]
QueenslandFLD_df.head(3)

,text,label
0,I just though about the night I went clubbing with @mikal1988 and I cried,not_relevant
1,Looks like its going to be another long night courtesy #djoko and #murray #AustralianOpen,not_relevant
2,@LaniiBanani hahahaha I just told him id have to think about it :p he was like what I would do to u if u were in my arms + more haha,not_relevant


### Number of hashtag characters

In [ ]:
# # calculating hashtags from NepalEQ_df Text Data
# NepalEQ_df['hashtags'] = NepalEQ_df['text'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
# NepalEQ_df.head()

In [ ]:
# # calculating hashtags from QueenslandFLD_df Text Data
# QueenslandFLD_df['hashtags'] = QueenslandFLD_df['text'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
# QueenslandFLD_df.head()

### Number of retweets

In [ ]:
# # calculating hashtags from NepalEQ_df Text Data
# NepalEQ_df['retweets'] = NepalEQ_df['text'].apply(lambda x: len([x for x in x.split() if x.startswith('RT')]))
# NepalEQ_df.head()

In [ ]:
# # calculating retweets from QueenslandFLD_df Text Data
# QueenslandFLD_df['retweets'] = QueenslandFLD_df['text'].apply(lambda x: len([x for x in x.split() if x.startswith('RT')]))
# QueenslandFLD_df.head()

### Converting to lower case

In [ ]:
# Converting NepalEQ_df Text Data to Lowercase
NepalEQ_df['text'] = NepalEQ_df['text'].apply(lambda x: " ".join(x.lower()
for x in x.split()))
NepalEQ_df['text'].head(3)

0    rt @anupkaphle: #nepal's prime minister addressed the country for 1st time since earthquake on saturday. no concrete plans, lots of referenâ°ã_
1                                              @jonsnowc4 so have we; read our friends blog from lamjung where they are working http://t.co/jgpsacuqpe
2                                                                                        lend a helping hand if you can #nepal https://t.co/fdrrvc84ea
Name: text, dtype: object

In [ ]:
# Converting NepalEQ_df Text Data to Lowercase
QueenslandFLD_df['text'] = QueenslandFLD_df['text'].apply(lambda x: " ".join(x.lower()
for x in x.split()))
QueenslandFLD_df['text'].head(3)

0                                                               i just though about the night i went clubbing with @mikal1988 and i cried
1                                               looks like its going to be another long night courtesy #djoko and #murray #australianopen
2    @laniibanani hahahaha i just told him id have to think about it :p he was like what i would do to u if u were in my arms + more haha
Name: text, dtype: object

### Removing URL links

In [ ]:
"""
Three expressions are used to remove URL links, one for URLs with https, the second for URLs with http, 
and the third for URLs without them, but with or without www.
"""

# Handling URL links for NepalEQ_df Text Data 
NepalEQ_df['text'] = NepalEQ_df['text'].apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
NepalEQ_df['text'] = NepalEQ_df['text'].apply(lambda x: re.sub(r'http?:\/\/\S+', '', x))
NepalEQ_df['text'] = NepalEQ_df['text'].apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
# Handling URL links for QueenslandFLD_df Text Data
QueenslandFLD_df['text'] = QueenslandFLD_df['text'].apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
QueenslandFLD_df['text'] = QueenslandFLD_df['text'].apply(lambda x: re.sub(r'http?:\/\/\S+', '', x))
QueenslandFLD_df['text'] = QueenslandFLD_df['text'].apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))

### Removing place holders 

In [ ]:
"""
Some text cleaning may have already been done on the dataset which replaced some 
links with {link} and all the videos with [video]. So, the code below is used to remove the {link} and [video] 
text in the tweets_text.
"""

# Handling place holders for QueenslandFLD_df Text Data
NepalEQ_df['text'] = NepalEQ_df['text'].apply(lambda x: re.sub(r'{link}', '', x))
NepalEQ_df['text'] = NepalEQ_df['text'].apply(lambda x: re.sub(r"\[video\]", '', x))
# Handling place holders for QueenslandFLD_df Text Data
QueenslandFLD_df['text'] = QueenslandFLD_df['text'].apply(lambda x: re.sub(r'{link}', '', x))
QueenslandFLD_df['text'] = QueenslandFLD_df['text'].apply(lambda x: re.sub(r"\[video\]", '', x))

### Removing HTML reference characters

In [ ]:
# Handling HTML reference characters for NepalEQ_df Text Data
NepalEQ_df['text'] = NepalEQ_df['text'].apply(lambda x: re.sub(r'&[a-z]+;', '', x))
# Handling HTML reference characters for QueenslandFLD_df Text Data
QueenslandFLD_df['text'] = QueenslandFLD_df['text'].apply(lambda x: re.sub(r'&[a-z]+;', '', x))

### Removing Non-Letter characters

In [ ]:
# Handling Non-Letter characters for NepalEQ_df Text Data
NepalEQ_df['text'] = NepalEQ_df['text'].apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))
# Handling Non-Letter characters for QueenslandFLD_df Text Data
QueenslandFLD_df['text'] = QueenslandFLD_df['text'].apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))

### Removing Twitter handles 

In [ ]:
# Handling Twitter handles for NepalEQ_df Text Data
NepalEQ_df['text'] = NepalEQ_df['text'].apply(lambda x: re.sub(r'@mention', '', x))
# Handling Twitter handles for QueenslandFLD_df Text Data
QueenslandFLD_df['text'] = QueenslandFLD_df['text'].apply(lambda x: re.sub(r'@mention', '', x))

### Stemming

In [ ]:
# """
# This is used to find the root of a word. For example, “fish,” “fishes,” and “fishing” are stemmed into fish.
# """
# # Initialize stem object
# st = PorterStemmer()

# # Stemming text for NepalEQ_df Text Data
# NepalEQ_df['text'] = NepalEQ_df['text'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
# # Stemming text for QueenslandFLD_df Text Data
# QueenslandFLD_df['text'] = QueenslandFLD_df['text'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

### Lemmatizing

In [ ]:
# """
# Lemmatization is a process of
# extracting a root word by considering the vocabulary. For example, “good”, “better”, or “best” is lemmatized into good.
# """

# # Lemmatizing text for NepalEQ_df Text Data
# NepalEQ_df['tokens'] = NepalEQ_df['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
# # Lemmatizing for QueenslandFLD_df Text Data
# QueenslandFLD_df['tokens'] = QueenslandFLD_df['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

### Tokenize the text

In [ ]:
# Initialize token object
tknzr = TweetTokenizer()

# Tokenize NepalEQ_df Text Data
NepalEQ_df['tokens'] = NepalEQ_df['text'].apply(tknzr.tokenize)
# Tokenize QueenslandFLD_df Text Data
QueenslandFLD_df['tokens'] = QueenslandFLD_df['text'].apply(tknzr.tokenize)

### Removing Punctuation

In [ ]:
# define punctuation list
PUNCUATION_LIST = list(string.punctuation)

# define punction function
def remove_punctuation(word_list):
    """Remove punctuation tokens from a list of tokens"""
    return [w for w in word_list if w not in PUNCUATION_LIST]

# Removing Punctuation for NepalEQ_df Token Data
NepalEQ_df['tokens'] = NepalEQ_df['tokens'].apply(remove_punctuation)
# Removing Punctuation for QueenslandFLD_df Token Data
QueenslandFLD_df['tokens'] = QueenslandFLD_df['tokens'].apply(remove_punctuation)

### Lemmatizing

In [ ]:
def lemmatize_word(text):
    """
        Lemmatize the tokenized words
    """

    lemmatizer = WordNetLemmatizer()
    lemma = [lemmatizer.lemmatize(word, tag) for word, tag in text]
    return lemma

# Test without POS Tagging
lemmatizer = WordNetLemmatizer()
# Lemmatizing text for NepalEQ_df Text Data
NepalEQ_df['lemmatize_word'] = NepalEQ_df['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# Lemmatizing for QueenslandFLD_df Text Data
QueenslandFLD_df['lemmatize_word'] = QueenslandFLD_df['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


In [ ]:
QueenslandFLD_df.head()

,text,label,tokens,lemmatize_word
0,i just though about the night i went clubbing with mikal and i cried,not_relevant,"[i, just, though, about, the, night, i, went, clubbing, with, mikal, and, i, cried]","[i, just, though, about, the, night, i, went, clubbing, with, mikal, and, i, cried]"
1,looks like its going to be another long night courtesy #djoko and #murray #australianopen,not_relevant,"[looks, like, its, going, to, be, another, long, night, courtesy, #djoko, and, #murray, #australianopen]","[look, like, it, going, to, be, another, long, night, courtesy, #djoko, and, #murray, #australianopen]"
2,laniibanani hahahaha i just told him id have to think about it :p he was like what i would do to u if u were in my arms more haha,not_relevant,"[laniibanani, hahahaha, i, just, told, him, id, have, to, think, about, it, :p, he, was, like, what, i, would, do, to, u, if, u, were, in, my, arms, more, haha]","[laniibanani, hahahaha, i, just, told, him, id, have, to, think, about, it, :p, he, wa, like, what, i, would, do, to, u, if, u, were, in, my, arm, more, haha]"
3,off to meeting with so called baaps of mineral processing,not_relevant,"[off, to, meeting, with, so, called, baaps, of, mineral, processing]","[off, to, meeting, with, so, called, baaps, of, mineral, processing]"
4,doubt i'll be getting much sleep tonight,not_relevant,"[doubt, i'll, be, getting, much, sleep, tonight]","[doubt, i'll, be, getting, much, sleep, tonight]"


## Feature engineering  

### Part of Speech Tagging (POS Tagging)

In [ ]:
wordnet_map = {"N":wordnet.NOUN, 
               "V":wordnet.VERB, 
               "J":wordnet.ADJ, 
               "R":wordnet.ADV
              }
    
train_sents = brown.tagged_sents(categories='news')
t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(train_sents, backoff=t0)
t2 = nltk.BigramTagger(train_sents, backoff=t1)

def pos_tag_wordnet(text, pos_tag_type="pos_tag"):
    """
        Create pos_tag with wordnet format
    """
    pos_tagged_text = t2.tag(text)
    
    # map the pos tagging output with wordnet output 
    pos_tagged_text = [(word, wordnet_map.get(pos_tag[0])) if pos_tag[0] in wordnet_map.keys() else (word, wordnet.NOUN) for (word, pos_tag) in pos_tagged_text ]
    return pos_tagged_text

NepalEQ_df['combined_postag_wnet'] = NepalEQ_df['lemmatize_word'].apply(lambda x: pos_tag_wordnet(x))
NepalEQ_df['joined_words'] = [' '.join(map(str, l)) for l in NepalEQ_df['lemmatize_word']] # join back to text
QueenslandFLD_df['combined_postag_wnet'] = QueenslandFLD_df['lemmatize_word'].apply(lambda x: pos_tag_wordnet(x))
QueenslandFLD_df['joined_words'] = [' '.join(map(str, l)) for l in QueenslandFLD_df['lemmatize_word']] # join back to text

In [ ]:
# Extract features based on the frequency of certain POS tags for NepalEQ_df
NepalEQ_df['Noun_Count'] = NepalEQ_df['combined_postag_wnet'].apply(lambda x: Counter(tag for word, tag in x)['n'])
NepalEQ_df['Verb_Count'] = NepalEQ_df['combined_postag_wnet'].apply(lambda x: Counter(tag for word, tag in x)['v'])
NepalEQ_df['Adjective_Count'] = NepalEQ_df['combined_postag_wnet'].apply(lambda x: Counter(tag for word, tag in x)['j'])
NepalEQ_df['Adverb_Count'] = NepalEQ_df['combined_postag_wnet'].apply(lambda x: Counter(tag for word, tag in x)['r'])

# Normalize the column
# norm1 = np.linalg.norm(NepalEQ_df['hashtags'])
# NepalEQ_df['hashtags_normalized'] = NepalEQ_df['hashtags'] / norm1
# norm2 = np.linalg.norm(NepalEQ_df['hashtags'])
# NepalEQ_df['retweets_normalized'] = NepalEQ_df['retweets'] / norm2

# norm3 = np.linalg.norm(NepalEQ_df['Noun_Count'])
# NepalEQ_df['Noun_Count_normalized'] = NepalEQ_df['Noun_Count'] / norm3
# norm4 = np.linalg.norm(NepalEQ_df['Verb_Count'])
# NepalEQ_df['Verb_Count_normalized'] = NepalEQ_df['Verb_Count'] / norm4
# norm5 = np.linalg.norm(NepalEQ_df['Adjective_Count'])
# NepalEQ_df['Adjective_Count_normalized'] = NepalEQ_df['Adjective_Count'] / norm5
# norm6 = np.linalg.norm(NepalEQ_df['Adverb_Count'])
# NepalEQ_df['Adverb_Count_normalized'] = NepalEQ_df['Adverb_Count'] / norm6

# NepalEQ_df['hashtags_normalized'] = NepalEQ_df['hashtags'].apply(lambda x: (x - NepalEQ_df['hashtags'].min()) / (NepalEQ_df['hashtags'].max() - NepalEQ_df['hashtags'].min()) * 2 - 1)
# NepalEQ_df['retweets_normalized'] = NepalEQ_df['retweets'].apply(lambda x: (x - NepalEQ_df['retweets'].min()) / (NepalEQ_df['retweets'].max() - NepalEQ_df['retweets'].min()) * 2 - 1)
# NepalEQ_df['Noun_Count_normalized'] = NepalEQ_df['Noun_Count'].apply(lambda x: (x - NepalEQ_df['Noun_Count'].min()) / (NepalEQ_df['Noun_Count'].max() - NepalEQ_df['Noun_Count'].min()) * 2 - 1)
# NepalEQ_df['Verb_Count_normalized'] = NepalEQ_df['Verb_Count'].apply(lambda x: (x - NepalEQ_df['Verb_Count'].min()) / (NepalEQ_df['Verb_Count'].max() - NepalEQ_df['Verb_Count'].min()) * 2 - 1)
# NepalEQ_df['Adjective_Count_normalized'] = NepalEQ_df['Adjective_Count'].apply(lambda x: (x - NepalEQ_df['Adjective_Count'].min()) / (NepalEQ_df['Adjective_Count'].max() - NepalEQ_df['Adjective_Count'].min()) * 2 - 1)
# NepalEQ_df['Adverb_Count_normalized'] = NepalEQ_df['Adverb_Count'].apply(lambda x: (x - NepalEQ_df['Adverb_Count'].min()) / (NepalEQ_df['Adverb_Count'].max() - NepalEQ_df['Adverb_Count'].min()) * 2 - 1)

# Extract features based on the frequency of certain POS tags for NepalEQ_df
QueenslandFLD_df['Noun_Count'] = QueenslandFLD_df['combined_postag_wnet'].apply(lambda x: Counter(tag for word, tag in x)['n'])
QueenslandFLD_df['Verb_Count'] = QueenslandFLD_df['combined_postag_wnet'].apply(lambda x: Counter(tag for word, tag in x)['v'])
QueenslandFLD_df['Adjective_Count'] = QueenslandFLD_df['combined_postag_wnet'].apply(lambda x: Counter(tag for word, tag in x)['j'])
QueenslandFLD_df['Adverb_Count'] = QueenslandFLD_df['combined_postag_wnet'].apply(lambda x: Counter(tag for word, tag in x)['r'])

# Normalize columns 
# norm7 = np.linalg.norm(QueenslandFLD_df['hashtags'])
# QueenslandFLD_df['hashtags_normalized'] = QueenslandFLD_df['hashtags'] / norm7
# norm8 = np.linalg.norm(QueenslandFLD_df['hashtags'])
# QueenslandFLD_df['retweets_normalized'] = QueenslandFLD_df['retweets'] / norm8

# norm9 = np.linalg.norm(QueenslandFLD_df['Noun_Count'])
# QueenslandFLD_df['Noun_Count_normalized'] = QueenslandFLD_df['Noun_Count'] / norm9
# norm10 = np.linalg.norm(QueenslandFLD_df['Verb_Count'])
# QueenslandFLD_df['Verb_Count_normalized'] = QueenslandFLD_df['Verb_Count'] / norm10
# norm11 = np.linalg.norm(QueenslandFLD_df['Adjective_Count'])
# QueenslandFLD_df['Adjective_Count_normalized'] = QueenslandFLD_df['Adjective_Count'] / norm11
# norm12 = np.linalg.norm(QueenslandFLD_df['Adverb_Count'])
# QueenslandFLD_df['Adverb_Count_normalized'] = QueenslandFLD_df['Adverb_Count'] / norm12

# # Normalize columns 
# QueenslandFLD_df['hashtags_normalized'] = QueenslandFLD_df['hashtags'].apply(lambda x: (x - QueenslandFLD_df['hashtags'].min()) / (QueenslandFLD_df['hashtags'].max() - QueenslandFLD_df['hashtags'].min()) * 2 - 1)
# QueenslandFLD_df['retweets_normalized'] = QueenslandFLD_df['retweets'].apply(lambda x: (x - QueenslandFLD_df['retweets'].min()) / (QueenslandFLD_df['retweets'].max() - QueenslandFLD_df['retweets'].min()) * 2 - 1)
# QueenslandFLD_df['Noun_Count_normalized'] = QueenslandFLD_df['Noun_Count'].apply(lambda x: (x - QueenslandFLD_df['Noun_Count'].min()) / (QueenslandFLD_df['Noun_Count'].max() - QueenslandFLD_df['Noun_Count'].min()) * 2 - 1)
# QueenslandFLD_df['Verb_Count_normalized'] = QueenslandFLD_df['Verb_Count'].apply(lambda x: (x - QueenslandFLD_df['Verb_Count'].min()) / (QueenslandFLD_df['Verb_Count'].max() - QueenslandFLD_df['Verb_Count'].min()) * 2 - 1)
# QueenslandFLD_df['Adjective_Count_normalized'] = QueenslandFLD_df['Adjective_Count'].apply(lambda x: (x - QueenslandFLD_df['Adjective_Count'].min()) / (QueenslandFLD_df['Adjective_Count'].max() - QueenslandFLD_df['Adjective_Count'].min()) * 2 - 1)
# QueenslandFLD_df['Adverb_Count_normalized'] = QueenslandFLD_df['Adverb_Count'].apply(lambda x: (x - QueenslandFLD_df['Adverb_Count'].min()) / (QueenslandFLD_df['Adverb_Count'].max() - QueenslandFLD_df['Adverb_Count'].min()) * 2 - 1)



In [ ]:
NepalEQ_df.head(2)

,text,label,tokens,lemmatize_word,combined_postag_wnet,joined_words,Noun_Count,Verb_Count,Adjective_Count,Adverb_Count
0,rt anupkaphle: #nepal's prime minister addressed the country for st time since earthquake on saturday no concrete plans lots of referen,relevant,"[rt, anupkaphle, #nepal's, prime, minister, addressed, the, country, for, st, time, since, earthquake, on, saturday, no, concrete, plans, lots, of, referen]","[rt, anupkaphle, #nepal's, prime, minister, addressed, the, country, for, st, time, since, earthquake, on, saturday, no, concrete, plan, lot, of, referen]","[(rt, n), (anupkaphle, n), (#nepal's, n), (prime, a), (minister, n), (addressed, v), (the, n), (country, n), (for, n), (st, n), (time, n), (since, n), (earthquake, n), (on, n), (saturday, n), (no, n), (concrete, a), (plan, n), (lot, n), (of, n), (referen, n)]",rt anupkaphle #nepal's prime minister addressed the country for st time since earthquake on saturday no concrete plan lot of referen,18,1,0,0
1,jonsnowc so have we; read our friends blog from lamjung where they are working,relevant,"[jonsnowc, so, have, we, read, our, friends, blog, from, lamjung, where, they, are, working]","[jonsnowc, so, have, we, read, our, friend, blog, from, lamjung, where, they, are, working]","[(jonsnowc, n), (so, n), (have, n), (we, n), (read, v), (our, n), (friend, n), (blog, n), (from, n), (lamjung, n), (where, n), (they, n), (are, n), (working, v)]",jonsnowc so have we read our friend blog from lamjung where they are working,12,2,0,0


In [ ]:
QueenslandFLD_df.head(2)

,text,label,tokens,lemmatize_word,combined_postag_wnet,joined_words,Noun_Count,Verb_Count,Adjective_Count,Adverb_Count
0,i just though about the night i went clubbing with mikal and i cried,not_relevant,"[i, just, though, about, the, night, i, went, clubbing, with, mikal, and, i, cried]","[i, just, though, about, the, night, i, went, clubbing, with, mikal, and, i, cried]","[(i, n), (just, r), (though, n), (about, r), (the, n), (night, n), (i, n), (went, v), (clubbing, n), (with, n), (mikal, n), (and, n), (i, n), (cried, n)]",i just though about the night i went clubbing with mikal and i cried,11,1,0,2
1,looks like its going to be another long night courtesy #djoko and #murray #australianopen,not_relevant,"[looks, like, its, going, to, be, another, long, night, courtesy, #djoko, and, #murray, #australianopen]","[look, like, it, going, to, be, another, long, night, courtesy, #djoko, and, #murray, #australianopen]","[(look, v), (like, n), (it, n), (going, v), (to, n), (be, n), (another, n), (long, a), (night, n), (courtesy, n), (#djoko, n), (and, n), (#murray, n), (#australianopen, n)]",look like it going to be another long night courtesy #djoko and #murray #australianopen,11,2,0,0


#### Hashtags

In [ ]:
# Extract hashtags from tweet text
NepalEQ_hashtags = NepalEQ_df['joined_words'].str.findall(r'#(\w+)')

# Apply one-hot encoding to hashtags
mlb = MultiLabelBinarizer()
one_hot = mlb.fit_transform(NepalEQ_hashtags)

# Create a new dataframe with one-hot encoded hashtag features
NepalEQ_hashtag_df = pd.DataFrame(one_hot, columns=mlb.classes_)

# Concatenate the original tweet dataframe with the new hashtag features dataframe
# NepalEQ_hashtag_df = pd.concat([tweets, hashtag_df], axis=1)

# View the resulting dataframe
NepalEQ_hashtag_df.head()

,aa,aamaadmiparty,aap,aapkilledfarmer,aapnautanki,aaprallymurder,abc,abdsc,abgurung,abitfurther,...,yogi,youcannotpreventearthquakes,youtube,ypg,ypj,yug,yunakim,yyc,zippednews,zuckerberg
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Extract hashtags from tweet text
QueenslandFLD_hashtags = QueenslandFLD_df['joined_words'].str.findall(r'#(\w+)')

# Apply one-hot encoding to hashtags
mlb = MultiLabelBinarizer()
one_hot = mlb.fit_transform(QueenslandFLD_hashtags)

# Create a new dataframe with one-hot encoded hashtag features
QueenslandFLD_hashtag_df = pd.DataFrame(one_hot, columns=mlb.classes_)

# Concatenate the original tweet dataframe with the new hashtag features dataframe
# NepalEQ_hashtag_df = pd.concat([tweets, hashtag_df], axis=1)

# View the resulting dataframe
QueenslandFLD_hashtag_df.head()

,aacta,abba,abcmobile,abcnews,ace,acnc,addicted,adi,adityaramadana,aftermath,...,year,yeerboy,yeppoon,yoga,yolo,younghealers,youreonlyonce,yuck,yumyum,zerodarkthirty
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Retweet

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Example tweet dataframe
#tweets = pd.DataFrame({'text': ['Just tweeted', 'RT @username: Some tweet', 'RT @otheruser: Another tweet']})

# Create a boolean mask indicating which tweets are retweets
NepalEQ_retweet = NepalEQ_df['joined_words'].str.startswith('RT')

# Apply one-hot encoding to the boolean mask
encoder = OneHotEncoder()
NepalEQ_retweet_encoded = encoder.fit_transform(NepalEQ_retweet.values.reshape(-1, 1)).toarray()

# Create a new dataframe with one-hot encoded retweet features
NepalEQ_retweet_df = pd.DataFrame(NepalEQ_retweet_encoded, columns=['is_retweet'])

# Concatenate the original tweet dataframe with the new retweet features dataframe
#NepalEQ_new_tweets = pd.concat([tweets, retweet_df], axis=1)

# View the resulting dataframe
NepalEQ_retweet_df.head()

,is_retweet
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Example tweet dataframe
#tweets = pd.DataFrame({'text': ['Just tweeted', 'RT @username: Some tweet', 'RT @otheruser: Another tweet']})

# Create a boolean mask indicating which tweets are retweets
QueenslandFLD_retweet = QueenslandFLD_df['joined_words'].str.startswith('RT')

# Apply one-hot encoding to the boolean mask
encoder = OneHotEncoder()
QueenslandFLD_retweet_encoded = encoder.fit_transform(QueenslandFLD_retweet.values.reshape(-1, 1)).toarray()

# Create a new dataframe with one-hot encoded retweet features
QueenslandFLD_retweet_df = pd.DataFrame(QueenslandFLD_retweet_encoded, columns=['is_retweet'])

# Concatenate the original tweet dataframe with the new retweet features dataframe
#NepalEQ_new_tweets = pd.concat([tweets, retweet_df], axis=1)

# View the resulting dataframe
QueenslandFLD_retweet_df.head()

,is_retweet
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


### Bag of words

In [ ]:
def cv(data, ngram = 1, MAX_NB_WORDS = 75000):
    count_vectorizer = CountVectorizer(ngram_range = (ngram, ngram), max_features = MAX_NB_WORDS)
    emb = count_vectorizer.fit_transform(data).toarray()
    print("count vectorize with", str(np.array(emb).shape[1]), "features")
    return emb, count_vectorizer

In [ ]:
# Print out for NepalEQ_df
def print_out2(emb, feat, ngram, compared_sentence=0):
    print(ngram,"bag-of-words: ")
    print(feat.get_feature_names(), "\n")
    print(ngram,"bag-of-feature: ")
    print(test_cv_1gram.vocabulary_, "\n")
    print("BoW matrix:")
    print(pd.DataFrame(emb.transpose(), index = feat.get_feature_names()).head(), "\n")
    print(ngram,"vector example:")
    print(NepalEQ_df['joined_words'][compared_sentence])
    print(emb[compared_sentence], "\n")
    
NepalEQ_test_corpus = NepalEQ_df['joined_words'][:5].tolist()
# print("The test corpus: ", test_corpus, "\n")

NepalEQ_test_cv_em_1gram, NepalEQ_test_cv_1gram = cv(NepalEQ_test_corpus, ngram=1)
# print_out(test_cv_em_1gram, test_cv_1gram, ngram="Uni-gram")

# NepalEQ_test_cv_em_2gram, NepalEQ_test_cv_2gram = cv(NepalEQ_test_corpus, ngram=2)
# # print_out(test_cv_em_2gram, test_cv_2gram, ngram="Bi-gram")

# NepalEQ_test_cv_em_3gram, NepalEQ_test_cv_3gram = cv(NepalEQ_test_corpus, ngram=3)
# # print_out(test_cv_em_2gram, test_cv_2gram, ngram="Tri-gram")

# implement into the whole NepalEQ_df
NepalEQ_df_corpus = NepalEQ_df['joined_words'].tolist()
NepalEQ_df_em_1gram, vc_1gram = cv(NepalEQ_df_corpus, 1)
# NepalEQ_df_em_2gram, vc_2gram = cv(NepalEQ_df_corpus, 2)
# NepalEQ_df_em_3gram, vc_3gram = cv(NepalEQ_df_corpus, 3)

print(len(NepalEQ_df_corpus))
print(NepalEQ_df_em_1gram.shape)
# print(NepalEQ_df_em_2gram.shape)
# print(NepalEQ_df_em_3gram.shape)

count vectorize with 66 features
count vectorize with 12109 features
6899
(6899, 12109)


In [ ]:
# Print out for QueenslandFLD_df
def print_out(emb, feat, ngram, compared_sentence=0):
    print(ngram,"bag-of-words: ")
    print(feat.get_feature_names(), "\n")
    print(ngram,"bag-of-feature: ")
    print(test_cv_1gram.vocabulary_, "\n")
    print("BoW matrix:")
    print(pd.DataFrame(emb.transpose(), index = feat.get_feature_names()).head(), "\n")
    print(ngram,"vector example:")
    print(QueenslandFLD_df['joined_words'][compared_sentence])
    print(emb[compared_sentence], "\n")
    
QueenslandFLD_test_corpus = QueenslandFLD_df['joined_words'][:5].tolist()
# print("The test corpus: ", test_corpus, "\n")

QueenslandFLD_test_cv_em_1gram, QueenslandFLD_test_cv_1gram = cv(QueenslandFLD_test_corpus, ngram=1)
# print_out(QueenslandFLD_test_cv_em_1gram, QueenslandFLD_test_cv_1gram, ngram="Uni-gram")

# QueenslandFLD_test_cv_em_2gram, QueenslandFLD_test_cv_2gram = cv(QueenslandFLD_test_corpus, ngram=2)
# # print_out(QueenslandFLD_test_cv_em_2gram, QueenslandFLD_test_cv_2gram, ngram="Bi-gram")

# QueenslandFLD_test_cv_em_3gram, QueenslandFLD_test_cv_3gram = cv(QueenslandFLD_test_corpus, ngram=3)
# # print_out(QueenslandFLD_test_cv_em_3gram, QueenslandFLD_test_cv_3gram, ngram="Tri-gram")

# implement into the whole NepalEQ_df
QueenslandFLD_df_corpus = QueenslandFLD_df['joined_words'].tolist()
QueenslandFLD_df_em_1gram, vc_1gram2 = cv(QueenslandFLD_df_corpus, 1)
# QueenslandFLD_df_em_2gram, vc_2gram2 = cv(QueenslandFLD_df_corpus, 2)
# QueenslandFLD_df_em_3gram, vc_3gram2 = cv(QueenslandFLD_df_corpus, 3)

print(len(QueenslandFLD_df_corpus))
print(QueenslandFLD_df_em_1gram.shape)
# print(QueenslandFLD_df_em_2gram.shape)
# print(QueenslandFLD_df_em_3gram.shape)

count vectorize with 56 features
count vectorize with 9587 features
6019
(6019, 9587)


### Term Frequency-Inverse Document Frequency (TF-IDF)

In [ ]:
def TFIDF(data, ngram = 1, MAX_NB_WORDS = 75000):
    tfidf_x = TfidfVectorizer(ngram_range = (ngram, ngram), max_features = MAX_NB_WORDS)
    emb = tfidf_x.fit_transform(data).toarray()
    print("tf-idf with", str(np.array(emb).shape[1]), "features")
    return emb, tfidf_x

In [ ]:
# TF-IDF for NepalEQ_df
NepalEQ_test_corpus = NepalEQ_df['joined_words'][:5].tolist()
#print("The test corpus: ", test_corpus, "\n")

NepalEQ_test_tfidf_em_1gram, NepalEQ_test_tfidf_1gram = TFIDF(NepalEQ_test_corpus, ngram=1)
#print_out2(NepalEQ_test_tfidf_em_1gram, NepalEQ_test_tfidf_1gram, ngram="Uni-gram")

# NepalEQ_test_tfidf_em_2gram, NepalEQ_test_tfidf_2gram = TFIDF(NepalEQ_test_corpus, ngram=2)
# #print_out2(NepalEQ_test_tfidf_em_2gram, NepalEQ_test_tfidf_2gram, ngram="Bi-gram")

# NepalEQ_test_tfidf_em_3gram, NepalEQ_test_tfidf_3gram = TFIDF(NepalEQ_test_corpus, ngram=3)
# #print_out2(NepalEQ_test_tfidf_em_3gram, NepalEQ_test_tfidf_3gram, ngram="Tri-gram")

# implement into the whole dataset
NepalEQ_df_corpus = NepalEQ_df['joined_words'].tolist()
NepalEQ_df_tfidf_1gram, tfidf_1gram = TFIDF(NepalEQ_df_corpus, 1)
# NepalEQ_df_tfidf_2gram, tfidf_2gram = TFIDF(NepalEQ_df_corpus, 2)
# NepalEQ_df_tfidf_3gram, tfidf_3gram = TFIDF(NepalEQ_df_corpus, 3)

print(len(NepalEQ_df_corpus))
print(NepalEQ_df_tfidf_1gram.shape)
# print(NepalEQ_df_tfidf_2gram.shape)
# print(NepalEQ_df_tfidf_3gram.shape)

tf-idf with 66 features
tf-idf with 12109 features
6899
(6899, 12109)


In [ ]:
# TF-IDF for QueenslandFLD_df
QueenslandFLD_test_corpus = QueenslandFLD_df['joined_words'][:5].tolist()
#print("The test corpus: ", test_corpus, "\n")

QueenslandFLD_test_tfidf_em_1gram, QueenslandFLD_test_tfidf_1gram = TFIDF(QueenslandFLD_test_corpus, ngram=1)
#print_out(QueenslandFLD_test_tfidf_em_1gram, QueenslandFLD_test_tfidf_1gram, ngram="Uni-gram")

# QueenslandFLD_test_tfidf_em_2gram, QueenslandFLD_test_tfidf_2gram = TFIDF(QueenslandFLD_test_corpus, ngram=2)
# #print_out(QueenslandFLD_test_tfidf_em_2gram, QueenslandFLD_test_tfidf_2gram, ngram="Bi-gram")

# QueenslandFLD_test_tfidf_em_3gram, QueenslandFLD_test_tfidf_3gram = TFIDF(QueenslandFLD_test_corpus, ngram=3)
# #print_out(NepalEQ_test_tfidf_em_3gram, NepalEQ_test_tfidf_3gram, ngram="Tri-gram")

# implement into the whole dataset
QueenslandFLD_df_corpus = QueenslandFLD_df['joined_words'].tolist()
QueenslandFLD_df_tfidf_1gram, tfidf_1gram2 = TFIDF(QueenslandFLD_df_corpus, 1)
# QueenslandFLD_df_tfidf_2gram, tfidf_2gram2 = TFIDF(QueenslandFLD_df_corpus, 2)
# QueenslandFLD_df_tfidf_3gram, tfidf_3gram2 = TFIDF(QueenslandFLD_df_corpus, 3)

print(len(QueenslandFLD_df_corpus))
print(QueenslandFLD_df_tfidf_1gram.shape)
# print(QueenslandFLD_df_tfidf_2gram.shape)
# print(QueenslandFLD_df_tfidf_3gram.shape)

tf-idf with 56 features
tf-idf with 9587 features
6019
(6019, 9587)


### Word2Vec

In [ ]:
# from pathlib import Path
# from google.colab import drive
# drive.mount('/content/drive')

word2vec_path = '/content/drive/MyDrive/GoogleNews-vectors-negative300.bin'

# we only load 200k most common words from Google News corpus
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True, limit=200000)

In [ ]:
# Compare the similarity between "cat" vs. "kitten" and "cat" vs. "cats"

print(word2vec_model.similarity('cat', 'kitten'))
print(word2vec_model.similarity('cat', 'cats'))

0.74649847
0.8099379


In [ ]:
def get_average_vec(tokens_list, vector, generate_missing=False, k=300):
    """
        Calculate average embedding value of sentence from each word vector
    """
    
    if len(tokens_list)<1:
        return np.zeros(k)
    
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_embeddings(vectors, text, generate_missing=False, k=300):
    """
        create the sentence embedding
    """
    embeddings = text.apply(lambda x: get_average_vec(x, vectors, generate_missing=generate_missing, k=k))
    return list(embeddings)

In [ ]:
# word2vec embedding for NepalEQ_df
NepalEQ_df_embeddings_word2vec = get_embeddings(word2vec_model, NepalEQ_df['joined_words'], k=300)

print("Embedding matrix size", len(NepalEQ_df_embeddings_word2vec), len(NepalEQ_df_embeddings_word2vec[0]))
print("The sentence: \"%s\" got embedding values: " % NepalEQ_df['joined_words'][0])
print(NepalEQ_df_embeddings_word2vec[0])

Embedding matrix size 6899 300
The sentence: "rt anupkaphle #nepal's prime minister addressed the country for st time since earthquake on saturday no concrete plan lot of referen" got embedding values: 
[-0.12967104  0.09333755 -0.01329457  0.11189131 -0.04398554  0.01486761
 -0.07004385 -0.02966679 -0.03154547  0.01759292 -0.04088751 -0.06536634
 -0.15724714  0.03127959 -0.09672778  0.08136264  0.08919363  0.13743638
 -0.01433586  0.02128092 -0.19123424 -0.03023541  0.08887528  0.00675672
 -0.07978081  0.02013524 -0.18579656  0.05401195 -0.02749171 -0.02586943
 -0.00714539  0.01675403 -0.072719   -0.08760857 -0.10657201  0.06748315
 -0.16937949  0.07192438 -0.04125468  0.05902608 -0.02513308 -0.05098378
  0.0348594   0.06829279  0.04072293 -0.02002736 -0.03920029 -0.13438092
 -0.1003418   0.07520086 -0.13920177  0.19218306 -0.03951564  0.16987564
  0.05048024  0.11500474 -0.12705855 -0.08820574  0.00082952 -0.13443456
 -0.0995562  -0.06259479 -0.14324674 -0.03739343 -0.04354419 -0.154

In [ ]:
# word2vec embedding for QueenslandFLD_df
QueenslandFLD_df_embeddings_word2vec = get_embeddings(word2vec_model, QueenslandFLD_df['joined_words'], k=300)

print("Embedding matrix size", len(QueenslandFLD_df_embeddings_word2vec), len(QueenslandFLD_df_embeddings_word2vec[0]))
print("The sentence: \"%s\" got embedding values: " % QueenslandFLD_df['joined_words'][0])
print(QueenslandFLD_df_embeddings_word2vec[0])

Embedding matrix size 6019 300
The sentence: "i just though about the night i went clubbing with mikal and i cried" got embedding values: 
[-1.63563448e-01  7.50436222e-02 -9.85448501e-03  1.10333611e-01
 -7.37286736e-02  1.06757669e-02 -8.07149550e-02 -6.53309542e-02
 -2.02914967e-02  7.56387150e-03 -4.45586934e-02 -9.07000373e-02
 -1.93838680e-01  6.78079269e-03 -9.35561236e-02  6.82624368e-02
  2.79763166e-02  1.38955509e-01 -1.81637932e-02 -1.38536341e-02
 -1.84408749e-01 -2.87531684e-02  1.03880714e-01 -6.31399716e-03
 -5.34367281e-02  3.72934762e-02 -2.33879538e-01  2.65915815e-02
 -8.75854492e-03 -9.51430377e-05  2.63501336e-02  2.92273129e-02
 -1.97516049e-02 -7.43848015e-02 -1.30762437e-01  8.94642998e-02
 -1.83158875e-01  1.07366225e-01 -4.87558140e-02  7.45957318e-02
 -1.99827306e-02 -4.13028493e-02  1.04440128e-01  7.65748865e-02
  4.69158958e-02 -3.87151382e-02  1.08041202e-02 -1.34212718e-01
 -7.94821347e-02  3.47684972e-02 -1.17438821e-01  1.69404871e-01
 -2.98982508e-02

### Global Vectors for Word Representation (GloVe):

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec

glove_input_file = "/content/drive/MyDrive/glove/glove.6B.300d.txt"
word2vec_output_file = "glove.6B.100d.txt.word2vec"
glove2word2vec(glove_input_file, word2vec_output_file)

# we only load 200k most common words from Google New corpus 
glove_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False, limit=200000) 


In [ ]:
# Compare the similarity between "cat" vs. "kitten" and "cat" vs. "cats" from GloVe

print(glove_model.similarity('cat', 'kitten'))
print(glove_model.similarity('cat', 'cats'))

0.43046176
0.68158364


In [ ]:
# GloVe embedding for NepalEQ_df
NepalEQ_df_embeddings_glove = get_embeddings(glove_model, NepalEQ_df['joined_words'], k=300)

print("Embedding matrix size", len(NepalEQ_df_embeddings_glove), len(NepalEQ_df_embeddings_glove[0]))
print("The sentence: \"%s\" got embedding values: " % NepalEQ_df['joined_words'][0])
print(NepalEQ_df_embeddings_glove[0])

Embedding matrix size 6899 300
The sentence: "rt anupkaphle #nepal's prime minister addressed the country for st time since earthquake on saturday no concrete plan lot of referen" got embedding values: 
[-2.28495274e-01 -6.08209594e-03 -2.15541902e-01 -3.94118740e-02
 -5.11049563e-01 -6.87474010e-03 -2.49461290e-02  1.65043709e-01
 -2.87330804e-01 -9.65558717e-01  1.99469061e-02  4.87846921e-02
 -2.94508145e-01  7.04633225e-02 -3.48408861e-02 -1.51335466e-01
 -2.39974160e-01  3.09178506e-01  1.62798942e-01 -1.48918035e-02
 -6.54065627e-02 -1.36595992e-01 -4.97086888e-02  5.38109673e-02
 -1.04650691e-01  1.33385234e-01  1.36464177e-01  1.80182668e-01
  3.42624088e-01 -3.47929564e-01 -9.67238854e-02  3.12049121e-01
 -2.19132944e-01 -1.22670792e-01 -6.06338107e-01  2.52418038e-02
  2.72592938e-01 -4.19366895e-01 -1.88869530e-01  1.20775761e-01
 -7.62118774e-02  6.64482197e-02 -4.71248429e-01  5.14597980e-01
 -2.43327927e-01 -2.27424684e-01  1.38591626e-01 -8.83835115e-02
  2.78771353e-02 

In [ ]:
# GloVe embedding for QueenslandFLD_df
QueenslandFLD_df_embeddings_glove = get_embeddings(glove_model, QueenslandFLD_df['joined_words'], k=300)

print("Embedding matrix size", len(QueenslandFLD_df_embeddings_glove), len(QueenslandFLD_df_embeddings_glove[0]))
print("The sentence: \"%s\" got embedding values: " % QueenslandFLD_df['joined_words'][0])
print(QueenslandFLD_df_embeddings_glove[0])

Embedding matrix size 6019 300
The sentence: "i just though about the night i went clubbing with mikal and i cried" got embedding values: 
[-0.180241    0.05367574 -0.22297876 -0.06396813 -0.50907456 -0.03861293
 -0.01607928  0.18043071 -0.28707592 -0.97813588  0.04812866 -0.02829128
 -0.3372479   0.09836832 -0.0438285  -0.08034226 -0.22341007  0.14967854
  0.16738716 -0.06665697 -0.00550996 -0.07034315 -0.03217519 -0.00369645
 -0.10867484  0.05491472  0.12675622  0.14748965  0.21412467 -0.27017695
 -0.13606109  0.25332606 -0.2670711  -0.11544512 -0.64905941  0.05140057
  0.17464412 -0.36758426 -0.14558212  0.12436544 -0.11094847 -0.06671769
 -0.45691253  0.36810354 -0.18775615 -0.20758322  0.20938235 -0.16841442
  0.00454974 -0.00662629  0.02318772  0.1114213  -0.05654474  0.06435995
 -0.02248227  0.29823227  0.168935   -0.02039458  0.07767013 -0.24694474
  0.45494457  0.21746578  0.01598245  0.06547459 -0.0469127  -0.0907941
 -0.21906404  0.10436953  0.22956766 -0.13917463 -0.1588529

### FastText

In [ ]:
from gensim.models.fasttext import FastText

fasttext_path = "/content/drive/MyDrive/wiki-news-300d-1M.vec"
fasttext_model = gensim.models.KeyedVectors.load_word2vec_format(fasttext_path, binary=False, limit=200000)

In [ ]:
print(fasttext_model.similarity('cat', 'kitten'))
print(fasttext_model.similarity('cat', 'cats'))

0.7353649
0.85528094


In [ ]:

# FastText embedding for NepalEQ_df
NepalEQ_df_embeddings_fasttext = get_embeddings(fasttext_model, NepalEQ_df['joined_words'], k=300)

print("Embedding matrix size", len(NepalEQ_df_embeddings_fasttext), len(NepalEQ_df_embeddings_fasttext[0]))
print("The sentence: \"%s\" got embedding values: " % NepalEQ_df['joined_words'][0])
print(NepalEQ_df_embeddings_fasttext[0])

Embedding matrix size 6899 300
The sentence: "rt anupkaphle #nepal's prime minister addressed the country for st time since earthquake on saturday no concrete plan lot of referen" got embedding values: 
[-9.07727273e-02  1.13992423e-02 -9.39007553e-02 -4.79871207e-02
  2.69871217e-02  4.32454546e-02  8.34621280e-03 -4.71227274e-02
  1.11727273e-01  3.30484841e-02 -1.94318183e-02 -3.49818182e-02
  4.80295450e-02 -2.83098483e-02 -1.85045459e-02  3.59848488e-02
 -1.71628779e-02 -2.64446969e-02 -5.75151503e-03  7.22492403e-02
 -5.04318196e-03 -1.28171969e-01 -2.60765150e-02 -1.38750005e-02
 -8.42931821e-02 -5.55265157e-02  4.31492427e-02  1.51395453e-01
  4.52098485e-02 -9.33537869e-02  4.26818194e-02  2.43075746e-02
  6.92803108e-03 -2.93204540e-02  7.45772728e-02  5.68886354e-02
 -1.30621214e-02  1.94674235e-02  5.18636369e-02 -2.06196974e-02
 -8.90757573e-03  9.03863627e-03 -7.20340901e-02 -1.51439347e-03
  2.22522730e-02  1.49712128e-02  3.63431829e-02  2.55363630e-02
  8.10522721e-02 

In [ ]:
# FastText embedding for QueenslandFLD_df
QueenslandFLD_df_embeddings_fasttext = get_embeddings(fasttext_model, QueenslandFLD_df['joined_words'], k=300)

print("Embedding matrix size", len(QueenslandFLD_df_embeddings_fasttext), len(QueenslandFLD_df_embeddings_fasttext[0]))
print("The sentence: \"%s\" got embedding values: " % QueenslandFLD_df['joined_words'][0])
print(QueenslandFLD_df_embeddings_fasttext[0])

Embedding matrix size 6019 300
The sentence: "i just though about the night i went clubbing with mikal and i cried" got embedding values: 
[-9.66705884e-02  2.84058822e-02 -6.99882343e-02 -3.63455885e-02
  3.88132357e-02  3.05161761e-02  2.02455889e-02 -1.55852946e-02
  1.28619119e-01  3.68249993e-02 -1.20161768e-02 -3.81323521e-02
  3.56088231e-02 -3.37058827e-02 -4.81750011e-02  3.66794122e-02
 -1.85617644e-02 -2.64264704e-02 -2.05367648e-02  5.61985281e-02
 -3.16911706e-03 -1.14554412e-01 -3.04691177e-02 -7.48529448e-04
 -7.21117642e-02 -6.65485294e-02  3.12132355e-02  1.75638233e-01
  6.18338245e-02 -9.11897057e-02  6.80058822e-02 -2.42205959e-03
  3.24220597e-02  1.88235014e-04  8.48794116e-02  4.20941176e-02
 -1.24676471e-02  5.02705869e-02  5.82985304e-02 -4.36176513e-03
 -9.03529401e-03  2.07661762e-02 -6.42897053e-02  1.45029413e-02
  1.02573528e-02  3.02352954e-02 -2.11838234e-02  5.55220580e-02
  9.61955886e-02 -1.26323518e-03 -2.30691176e-02 -4.45279410e-02
 -5.61247057e-01

### BERT

In [ ]:
import tensorflow_hub as hub

# download the tonkenizer 
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
#!pip3 install tokenization
#import tokenization
#!pip install bert-tensorflow
#from bert import tokenization
#from bert import bert_tokenization
!pip install --upgrade transformers
from transformers import BertTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 17.8 MB/s eta 0:00:00


In [ ]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [ ]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
#tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)
tokenizer = BertTokenizer(vocab_file, do_lower_case)

# BERT for NepalEQ_df
NepalEQ_df_bert_input = bert_encode(NepalEQ_df['joined_words'].values, tokenizer, max_len=200)

# BERT for QueenslandFLD_df
QueenslandFLD_df_bert_input = bert_encode(QueenslandFLD_df['joined_words'].values, tokenizer, max_len=200)

In [ ]:
print("Embedding tensor size", len(NepalEQ_df_bert_input), len(NepalEQ_df_bert_input[0]), len(NepalEQ_df_bert_input[0][0]))
#print("The sentence: \"%s\" got embedding values: " % NepalEQ_df_bert_input['joined_words'][0])
print(NepalEQ_df_bert_input[0])

Embedding tensor size 3 6899 200
[[  101 19387  2019 ...     0     0     0]
 [  101  6285  2015 ...     0     0     0]
 [  101 18496  1037 ...     0     0     0]
 ...
 [  101 11968 26095 ...     0     0     0]
 [  101  3892  2006 ...     0     0     0]
 [  101 10905  7630 ...     0     0     0]]


In [ ]:
print("Embedding tensor size", len(QueenslandFLD_df_bert_input), len(QueenslandFLD_df_bert_input[0]), len(QueenslandFLD_df_bert_input[0][0]))
print("The sentence: \"%s\" got embedding values: " % QueenslandFLD_df['joined_words'][0])
print(QueenslandFLD_df_bert_input[0])

Embedding tensor size 3 6019 200
The sentence: "i just though about the night i went clubbing with mikal and i cried" got embedding values: 
[[  101  1045  2074 ...     0     0     0]
 [  101  2298  2066 ...     0     0     0]
 [  101 17595  6137 ...     0     0     0]
 ...
 [  101 19387  2739 ...     0     0     0]
 [  101 19387  5925 ...     0     0     0]
 [  101 19387  2188 ...     0     0     0]]


## Classification 

In [ ]:
# Replace the label column with 1's and 0's using pandas.replace
# NepalEQ_df['label'] = NepalEQ_df['label'].replace({'relevant': 1, 'not_relevant': 0})
# QueenslandFLD_df['label'] = QueenslandFLD_df['label'].replace({'relevant': 1, 'not_relevant': 0})

from sklearn.preprocessing import LabelEncoder

# Assume your label column is called 'class_label'
label_encoder = LabelEncoder()
NepalEQ_df['label'] = label_encoder.fit_transform(NepalEQ_df['label'])
QueenslandFLD_df['label'] = label_encoder.fit_transform(QueenslandFLD_df['label'])

np.save('NepalEQ_df_label.npy', NepalEQ_df.label)
np.save('QueenslandFLD_df_label.npy', QueenslandFLD_df.label)

### Concatenate all vectors

In [ ]:
NepalEQ_df.columns

Index(['text', 'label', 'tokens', 'lemmatize_word', 'combined_postag_wnet',
       'joined_words', 'Noun_Count', 'Verb_Count', 'Adjective_Count',
       'Adverb_Count'],
      dtype='object')

#### Hashtags

In [ ]:
# Convert the dataframe to a 2D array
NepalEQ_hashtag_arr = NepalEQ_hashtag_df.to_numpy()
NepalEQ_hashtag_arr_matrix = coo_matrix(NepalEQ_hashtag_arr).toarray()

# Convert the dataframe to a 2D array
QueenslandFLD_hashtag_arr = QueenslandFLD_hashtag_df.to_numpy()
QueenslandFLD_hashtag_arr_matrix = coo_matrix(QueenslandFLD_hashtag_arr).toarray()

# save to npy files
np.save('NepalEQ_hashtag_arr_matrix.npy', NepalEQ_hashtag_arr_matrix)
np.save('QueenslandFLD_hashtag_arr_matrix.npy', QueenslandFLD_hashtag_arr_matrix)

#### Retweets

In [ ]:
# Convert the dataframe to a 2D array
NepalEQ_retweet_arr = NepalEQ_retweet_df.to_numpy()
NepalEQ_retweet_arr_matrix = coo_matrix(NepalEQ_retweet_arr).toarray()

# Convert the dataframe to a 2D array
QueenslandFLD_retweet_arr = QueenslandFLD_retweet_df.to_numpy()
QueenslandFLD_retweet_arr_matrix = coo_matrix(QueenslandFLD_retweet_arr).toarray()

# save to npy files
np.save('NepalEQ_retweet_arr_matrix.npy', NepalEQ_retweet_arr_matrix)
np.save('QueenslandFLD_retweet_arr_matrix.npy', QueenslandFLD_retweet_arr_matrix)

#### BOW

In [ ]:
# Convert the dataframe to a 2D array
NepalEQ_df_em_1gram_matrix = coo_matrix(NepalEQ_df_em_1gram).toarray()

# Convert the dataframe to a 2D array
QueenslandFLD_df_em_1gram_matrix = coo_matrix(QueenslandFLD_df_em_1gram).toarray()

# save to npy files
np.save('NepalEQ_df_em_1gram_matrix.npy', NepalEQ_df_em_1gram_matrix)
np.save('QueenslandFLD_df_em_1gram_matrix.npy', QueenslandFLD_df_em_1gram_matrix)

#### POS

In [ ]:
# Convert the new DataFrame to a NumPy array for NepalEQ_df
NepalEQ_POS_selected = NepalEQ_df[['Noun_Count', 'Verb_Count', 'Adjective_Count', 'Adverb_Count']]
# Convert the dataframe to a 2D array
NepalEQ_POS_arr = NepalEQ_POS_selected.to_numpy()
NepalEQ_POS_matrix = coo_matrix(NepalEQ_POS_arr).toarray()

# Convert the new DataFrame to a NumPy array for QueenslandFLD_df
QueenslandFLD_POS_selected = QueenslandFLD_df[['Noun_Count', 'Verb_Count', 'Adjective_Count', 'Adverb_Count']]
# Convert the dataframe to a 2D array
QueenslandFLD_df_POS_arr = QueenslandFLD_POS_selected.to_numpy()
QueenslandFLD_POS_matrix = coo_matrix(QueenslandFLD_df_POS_arr).toarray()

# save to npy files
np.save('NepalEQ_POS_matrix.npy', NepalEQ_POS_matrix)
np.save('QueenslandFLD_POS_matrix.npy', QueenslandFLD_POS_matrix)

#### Fasttext

In [ ]:
# Create a sparse matrix in COO format NepalEQ_df_embeddings_fasttext
NepalEQ_df_embeddings_fasttext_matrix = coo_matrix(NepalEQ_df_embeddings_fasttext).toarray()

# Create a sparse matrix in COO format QueenslandFLD_df_embeddings_fasttext
QueenslandFLD_df_embeddings_fasttext_matrix = coo_matrix(QueenslandFLD_df_embeddings_fasttext).toarray()

# save to npy files
np.save('NepalEQ_df_embeddings_fasttext_matrix.npy', NepalEQ_df_embeddings_fasttext_matrix)
np.save('QueenslandFLD_df_embeddings_fasttext_matrix.npy', QueenslandFLD_df_embeddings_fasttext_matrix)

#### Word2Vec

In [ ]:
# Create a sparse matrix in COO format NepalEQ_df_embeddings_word2vec
NepalEQ_df_embeddings_word2vec_matrix = coo_matrix(NepalEQ_df_embeddings_word2vec).toarray()

# Create a sparse matrix in COO format QueenslandFLD_df_embeddings_word2vec
QueenslandFLD_df_embeddings_word2vec_matrix = coo_matrix(QueenslandFLD_df_embeddings_word2vec).toarray()

# save to npy files
np.save('NepalEQ_df_embeddings_word2vec_matrix.npy', NepalEQ_df_embeddings_word2vec_matrix)
np.save('QueenslandFLD_df_embeddings_word2vec_matrix.npy', QueenslandFLD_df_embeddings_word2vec_matrix)

#### Glove

In [ ]:
# Create a sparse matrix in COO format NepalEQ_df_embeddings_glove
NepalEQ_df_embeddings_glove_matrix = coo_matrix(NepalEQ_df_embeddings_glove[0]).toarray()

# Create a sparse matrix in COO format QueenslandFLD_df_embeddings_glove
QueenslandFLD_df_embeddings_glove_matrix = coo_matrix(QueenslandFLD_df_embeddings_glove[0]).toarray()

# save to npy files
np.save('NepalEQ_df_embeddings_glove_matrix.npy', NepalEQ_df_embeddings_glove_matrix)
np.save('QueenslandFLD_df_embeddings_glove_matrix.npy', QueenslandFLD_df_embeddings_glove_matrix)

#### Bert

In [ ]:
# Create a sparse matrix in COO format NepalEQ_df_bert_input
NepalEQ_df_bert_input_matrix = coo_matrix(NepalEQ_df_bert_input).toarray()

# Create a sparse matrix in COO format QueenslandFLD_df_embeddings_glove
QueenslandFLD_df_bert_input_matrix = coo_matrix(QueenslandFLD_df_bert_input).toarray()

# save to npy files
np.save('NepalEQ_df_bert_input_matrix.npy', NepalEQ_df_bert_input_matrix)
np.save('QueenslandFLD_df_bert_input_matrix.npy', QueenslandFLD_df_bert_input_matrix)

TypeError: ignored

#### TF-IDF

In [ ]:
# Convert the dataframe to a 2D array
NepalEQ_df_tfidf_1gram_matrix = coo_matrix(NepalEQ_df_tfidf_1gram).toarray()

# Convert the dataframe to a 2D array
QueenslandFLD_df_tfidf_1gram_matrix = coo_matrix(QueenslandFLD_df_tfidf_1gram).toarray()

# save to npy files
np.save('NepalEQ_df_tfidf_1gram_matrix.npy', NepalEQ_df_tfidf_1gram_matrix)
np.save('QueenslandFLD_df_tfidf_1gram_matrix.npy', QueenslandFLD_df_tfidf_1gram_matrix)

#### TF-IDF + hashtags + Retweets + POS + FT

In [ ]:
NepalEQ_df_tfidf_1gram_matrix2 = coo_matrix(NepalEQ_df_tfidf_1gram)
NepalEQ_hashtag_arr_matrix2 = coo_matrix(NepalEQ_hashtag_arr)
NepalEQ_retweet_arr_matrix2 = coo_matrix(NepalEQ_retweet_arr)
NepalEQ_POS_matrix2 = coo_matrix(NepalEQ_POS_arr)
NepalEQ_df_embeddings_fasttext_matrix2 = coo_matrix(NepalEQ_df_embeddings_fasttext)
NepalEQ_TFIDF_HT_RT_POS_FT_hstack = hstack([NepalEQ_df_tfidf_1gram_matrix2, 
                                    NepalEQ_hashtag_arr_matrix2, 
                                    NepalEQ_retweet_arr_matrix2, 
                                    NepalEQ_POS_matrix2, 
                                    NepalEQ_df_embeddings_fasttext_matrix2]).toarray()


QueenslandFLD_df_tfidf_1gram_matrix2 = coo_matrix(QueenslandFLD_df_tfidf_1gram)
QueenslandFLD_hashtag_arr_matrix2 = coo_matrix(QueenslandFLD_hashtag_arr)
QueenslandFLD_retweet_arr_matrix2 = coo_matrix(QueenslandFLD_retweet_arr)
QueenslandFLD_POS_matrix2 = coo_matrix(QueenslandFLD_df_POS_arr)
QueenslandFLD_df_embeddings_fasttext_matrix2 = coo_matrix(QueenslandFLD_df_embeddings_fasttext)
QueenslandFLD_TFIDF_HT_RT_POS_FT_hstack = hstack([QueenslandFLD_df_tfidf_1gram_matrix2, 
                                    QueenslandFLD_hashtag_arr_matrix2, 
                                    QueenslandFLD_retweet_arr_matrix2, 
                                    QueenslandFLD_POS_matrix2, 
                                    QueenslandFLD_df_embeddings_fasttext_matrix2]).toarray()


# save to npy files
np.save('NepalEQ_TFIDF_HT_RT_POS_FT_hstack.npy', NepalEQ_TFIDF_HT_RT_POS_FT_hstack)
np.save('QueenslandFLD_TFIDF_HT_RT_POS_FT_hstack.npy', QueenslandFLD_TFIDF_HT_RT_POS_FT_hstack)                                    

In [ ]:
QueenslandFLD_hashtag_arr_matrix2.shape

#### TF-IDF + hashtags + Retweets + POS + word2vec

In [ ]:
NepalEQ_df_tfidf_1gram_matrix2 = coo_matrix(NepalEQ_df_tfidf_1gram)
NepalEQ_hashtag_arr_matrix2 = coo_matrix(NepalEQ_hashtag_arr)
NepalEQ_retweet_arr_matrix2 = coo_matrix(NepalEQ_retweet_arr)
NepalEQ_POS_matrix2 = coo_matrix(NepalEQ_POS_arr)
NepalEQ_df_embeddings_word2vec_matrix2 = coo_matrix(NepalEQ_df_embeddings_word2vec)
NepalEQ_TFIDF_HT_RT_POS_w2v_hstack = hstack([NepalEQ_df_tfidf_1gram_matrix2, 
                                              NepalEQ_hashtag_arr_matrix2, 
                                              NepalEQ_retweet_arr_matrix2, 
                                              NepalEQ_POS_matrix2, 
                                              NepalEQ_df_embeddings_word2vec_matrix2]).toarray()


QueenslandFLD_df_tfidf_1gram_matrix2 = coo_matrix(QueenslandFLD_df_tfidf_1gram)
QueenslandFLD_hashtag_arr_matrix2 = coo_matrix(QueenslandFLD_hashtag_arr)
QueenslandFLD_retweet_arr_matrix2 = coo_matrix(QueenslandFLD_retweet_arr)
QueenslandFLD_POS_matrix2 = coo_matrix(QueenslandFLD_df_POS_arr)
QueenslandFLD_df_embeddings_word2vec_matrix2 = coo_matrix(QueenslandFLD_df_embeddings_word2vec)
QueenslandFLD_TFIDF_HT_RT_POS_w2v_hstack = hstack([QueenslandFLD_df_tfidf_1gram_matrix2, 
                                                    QueenslandFLD_hashtag_arr_matrix2, 
                                                    QueenslandFLD_retweet_arr_matrix2, 
                                                    QueenslandFLD_POS_matrix2, 
                                                    QueenslandFLD_df_embeddings_word2vec_matrix2]).toarray()


# save to npy files
np.save('NepalEQ_TFIDF_HT_RT_POS_w2v_hstack.npy', NepalEQ_TFIDF_HT_RT_POS_w2v_hstack)
np.save('QueenslandFLD_TFIDF_HT_RT_POS_w2v_hstack.npy', QueenslandFLD_TFIDF_HT_RT_POS_w2v_hstack)                                                     

#### TF-IDF + hashtags + Retweets + POS + Glove

In [ ]:
NepalEQ_df_tfidf_1gram_matrix2 = coo_matrix(NepalEQ_df_tfidf_1gram)
NepalEQ_hashtag_arr_matrix2 = coo_matrix(NepalEQ_hashtag_arr)
NepalEQ_retweet_arr_matrix2 = coo_matrix(NepalEQ_retweet_arr)
NepalEQ_POS_matrix2 = coo_matrix(NepalEQ_POS_arr)
NepalEQ_df_embeddings_glove_matrix2 = coo_matrix(NepalEQ_df_embeddings_glove)
NepalEQ_TFIDF_HT_RT_POS_glove_hstack = hstack([NepalEQ_df_tfidf_1gram_matrix2, 
                                     NepalEQ_hashtag_arr_matrix2, 
                                     NepalEQ_retweet_arr_matrix2, 
                                     NepalEQ_POS_matrix2, 
                                     NepalEQ_df_embeddings_glove_matrix2]).toarray()


QueenslandFLD_df_tfidf_1gram_matrix2 = coo_matrix(QueenslandFLD_df_tfidf_1gram)
QueenslandFLD_hashtag_arr_matrix2 = coo_matrix(QueenslandFLD_hashtag_arr)
QueenslandFLD_retweet_arr_matrix2 = coo_matrix(QueenslandFLD_retweet_arr)
QueenslandFLD_POS_matrix2 = coo_matrix(QueenslandFLD_df_POS_arr)
QueenslandFLD_df_embeddings_glove_matrix2 = coo_matrix(QueenslandFLD_df_embeddings_glove)
QueenslandFLD_TFIDF_HT_RT_POS_glove_hstack = hstack([QueenslandFLD_df_tfidf_1gram_matrix2, 
                                     QueenslandFLD_hashtag_arr_matrix2, 
                                     QueenslandFLD_retweet_arr_matrix2, 
                                     QueenslandFLD_POS_matrix2, 
                                     QueenslandFLD_df_embeddings_glove_matrix2]).toarray()


# save to npy files
np.save('NepalEQ_TFIDF_HT_RT_POS_glove_hstack.npy', NepalEQ_TFIDF_HT_RT_POS_glove_hstack)
np.save('QueenslandFLD_TFIDF_HT_RT_POS_glove_hstack.npy', QueenslandFLD_TFIDF_HT_RT_POS_glove_hstack)                                     

#### TF-IDF + hashtags + Retweets + POS + bert

In [ ]:
NepalEQ_df_tfidf_1gram_matrix2 = coo_matrix(NepalEQ_df_tfidf_1gram)
NepalEQ_hashtag_arr_matrix2 = coo_matrix(NepalEQ_hashtag_arr)
NepalEQ_retweet_arr_matrix2 = coo_matrix(NepalEQ_retweet_arr)
NepalEQ_POS_matrix2 = coo_matrix(NepalEQ_POS_arr)
NepalEQ_df_bert_input_matrix2 = coo_matrix(NepalEQ_df_bert_input[0])
NepalEQ_TFIDF_HT_RT_POS_bert_hstack = hstack([NepalEQ_df_tfidf_1gram_matrix2, 
                                     NepalEQ_hashtag_arr_matrix2, 
                                     NepalEQ_retweet_arr_matrix2, 
                                     NepalEQ_POS_matrix2, 
                                     NepalEQ_df_bert_input_matrix2]).toarray()


QueenslandFLD_df_tfidf_1gram_matrix2 = coo_matrix(QueenslandFLD_df_tfidf_1gram)
QueenslandFLD_hashtag_arr_matrix2 = coo_matrix(QueenslandFLD_hashtag_arr)
QueenslandFLD_retweet_arr_matrix2 = coo_matrix(QueenslandFLD_retweet_arr)
QueenslandFLD_POS_matrix2 = coo_matrix(QueenslandFLD_df_POS_arr)
QueenslandFLD_df_bert_input_matrix2 = coo_matrix(QueenslandFLD_df_bert_input[0])
QueenslandFLD_TFIDF_HT_RT_POS_bert_hstack = hstack([QueenslandFLD_df_tfidf_1gram_matrix2, 
                                     QueenslandFLD_hashtag_arr_matrix2, 
                                     QueenslandFLD_retweet_arr_matrix2, 
                                     QueenslandFLD_POS_matrix2, 
                                     QueenslandFLD_df_bert_input_matrix2]).toarray()



# save to npy files
np.save('NepalEQ_TFIDF_HT_RT_POS_bert_hstack.npy', NepalEQ_TFIDF_HT_RT_POS_bert_hstack)
np.save('QueenslandFLD_TFIDF_HT_RT_POS_bert_hstack.npy', QueenslandFLD_TFIDF_HT_RT_POS_bert_hstack)                                    

#### hastags + POS + TF-IDF

In [ ]:
NepalEQ_df_tfidf_1gram_matrix2 = coo_matrix(NepalEQ_df_tfidf_1gram)
NepalEQ_hashtag_arr_matrix2 = coo_matrix(NepalEQ_hashtag_arr)
NepalEQ_POS_matrix2 = coo_matrix(NepalEQ_POS_arr)
NepalEQ_TFIDF_HT_POS_hstack = hstack([NepalEQ_df_tfidf_1gram_matrix2, 
                                     NepalEQ_hashtag_arr_matrix2,  
                                     NepalEQ_POS_matrix2]).toarray()

QueenslandFLD_df_tfidf_1gram_matrix2 = coo_matrix(QueenslandFLD_df_tfidf_1gram)
QueenslandFLD_hashtag_arr_matrix2 = coo_matrix(QueenslandFLD_hashtag_arr)
QueenslandFLD_POS_matrix2 = coo_matrix(QueenslandFLD_df_POS_arr)
QueenslandFLD_TFIDF_HT_POS_hstack = hstack([QueenslandFLD_df_tfidf_1gram_matrix2, 
                                            QueenslandFLD_hashtag_arr_matrix2, 
                                            QueenslandFLD_POS_matrix2]).toarray()



# save to npy files
np.save('NepalEQ_TFIDF_HT_POS_hstack.npy', NepalEQ_TFIDF_HT_POS_hstack)
np.save('QueenslandFLD_TFIDF_HT_POS_hstack.npy', QueenslandFLD_TFIDF_HT_POS_hstack)                                    

#### hastags + POS + TF-IDF + BERT

In [ ]:
NepalEQ_df_tfidf_1gram_matrix2 = coo_matrix(NepalEQ_df_tfidf_1gram)
NepalEQ_hashtag_arr_matrix2 = coo_matrix(NepalEQ_hashtag_arr)
NepalEQ_POS_matrix2 = coo_matrix(NepalEQ_POS_arr)
NepalEQ_df_bert_input_matrix2 = coo_matrix(NepalEQ_df_bert_input[0])
NepalEQ_TFIDF_HT_POS_BERT_hstack = hstack([NepalEQ_df_tfidf_1gram_matrix2, 
                                           NepalEQ_hashtag_arr_matrix2, 
                                           NepalEQ_POS_matrix2, 
                                           NepalEQ_df_bert_input_matrix2]).toarray()

QueenslandFLD_df_tfidf_1gram_matrix2 = coo_matrix(QueenslandFLD_df_tfidf_1gram)
QueenslandFLD_hashtag_arr_matrix2 = coo_matrix(QueenslandFLD_hashtag_arr)
QueenslandFLD_POS_matrix2 = coo_matrix(QueenslandFLD_df_POS_arr)
QueenslandFLD_df_bert_input_matrix2 = coo_matrix(QueenslandFLD_df_bert_input[0])
QueenslandFLD_TFIDF_HT_POS_BERT_hstack = hstack([QueenslandFLD_df_tfidf_1gram_matrix2, 
                                                 QueenslandFLD_hashtag_arr_matrix2, 
                                                 QueenslandFLD_POS_matrix2, 
                                                 QueenslandFLD_df_bert_input_matrix2]).toarray()



# save to npy files
np.save('NepalEQ_TFIDF_HT_POS_BERT_hstack.npy', NepalEQ_TFIDF_HT_POS_BERT_hstack)
np.save('QueenslandFLD_TFIDF_HT_POS_BERT_hstack.npy', QueenslandFLD_TFIDF_HT_POS_BERT_hstack)     

#### hastags + POS + TF-IDF + GLoVe

In [ ]:
NepalEQ_df_tfidf_1gram_matrix2 = coo_matrix(NepalEQ_df_tfidf_1gram)
NepalEQ_hashtag_arr_matrix2 = coo_matrix(NepalEQ_hashtag_arr)
NepalEQ_POS_matrix2 = coo_matrix(NepalEQ_POS_arr)
NepalEQ_df_embeddings_glove_matrix2 = coo_matrix(NepalEQ_df_embeddings_glove)
NepalEQ_TFIDF_HT_POS_GLOVE_hstack = hstack([NepalEQ_df_tfidf_1gram_matrix2, 
                                           NepalEQ_hashtag_arr_matrix2, 
                                           NepalEQ_POS_matrix2, 
                                           NepalEQ_df_embeddings_glove_matrix2]).toarray()

QueenslandFLD_df_tfidf_1gram_matrix2 = coo_matrix(QueenslandFLD_df_tfidf_1gram)
QueenslandFLD_hashtag_arr_matrix2 = coo_matrix(QueenslandFLD_hashtag_arr)
QueenslandFLD_POS_matrix2 = coo_matrix(QueenslandFLD_df_POS_arr)
QueenslandFLD_df_embeddings_glove_matrix2 = coo_matrix(QueenslandFLD_df_embeddings_glove)
QueenslandFLD_TFIDF_HT_POS_GLOVE_hstack = hstack([QueenslandFLD_df_tfidf_1gram_matrix2, 
                                                 QueenslandFLD_hashtag_arr_matrix2, 
                                                 QueenslandFLD_POS_matrix2, 
                                                 QueenslandFLD_df_embeddings_glove_matrix2]).toarray()



# save to npy files
np.save('NepalEQ_TFIDF_HT_POS_GLOVE_hstack.npy', NepalEQ_TFIDF_HT_POS_GLOVE_hstack)
np.save('QueenslandFLD_TFIDF_HT_POS_GLOVE_hstack.npy', QueenslandFLD_TFIDF_HT_POS_GLOVE_hstack)  

#### TF-IDF + hashtags + Retweets

In [ ]:
# # Convert the new DataFrame to a NumPy array for NepalEQ_df
# NepalEQ_df_selected = NepalEQ_df[['hashtags_normalized','retweets_normalized']]
# # Convert the dataframe to a 2D array
# NepalEQ_arr = NepalEQ_df_selected.to_numpy()

# # Create a sparse matrix in COO format
# NepalEQ_arr_matrix = coo_matrix(NepalEQ_arr)
# NepalEQ_df_tfidf_1gram_matrix = coo_matrix(NepalEQ_df_tfidf_1gram)
# NepalEQ_df_tfidf_1gram_hstack = hstack([NepalEQ_arr_matrix,NepalEQ_df_tfidf_1gram_matrix]).toarray()

# # NepalEQ_arr_matrix = coo_matrix(NepalEQ_arr)
# # NepalEQ_df_tfidf_2gram_matrix = coo_matrix(NepalEQ_df_tfidf_2gram)
# # NepalEQ_df_tfidf_2gram_hstack = hstack([NepalEQ_arr_matrix,NepalEQ_df_tfidf_2gram_matrix]).toarray()

# # NepalEQ_arr_matrix = coo_matrix(NepalEQ_arr)
# # NepalEQ_df_tfidf_3gram_matrix = coo_matrix(NepalEQ_df_tfidf_3gram)
# # NepalEQ_df_tfidf_3gram_hstack = hstack([NepalEQ_arr_matrix,NepalEQ_df_tfidf_3gram_matrix]).toarray()


# # Convert the new DataFrame to a NumPy array for QueenslandFLD_df
# QueenslandFLD_df_selected = QueenslandFLD_df[['hashtags', 'retweets']]
# # Convert the dataframe to a 2D array
# QueenslandFLD_arr = QueenslandFLD_df_selected.to_numpy()

# # Create a sparse matrix in COO format
# QueenslandFLD_arr_matrix = coo_matrix(QueenslandFLD_arr)
# QueenslandFLD_df_tfidf_1gram_matrix = coo_matrix(QueenslandFLD_df_tfidf_1gram)
# QueenslandFLD_df_tfidf_1gram_hstack = hstack([QueenslandFLD_arr_matrix,QueenslandFLD_df_tfidf_1gram_matrix]).toarray()

# # QueenslandFLD_arr_matrix = coo_matrix(QueenslandFLD_arr)
# # QueenslandFLD_df_tfidf_2gram_matrix = coo_matrix(QueenslandFLD_df_tfidf_2gram)
# # QueenslandFLD_df_tfidf_2gram_hstack = hstack([QueenslandFLD_arr_matrix,QueenslandFLD_df_tfidf_2gram_matrix]).toarray()

# # QueenslandFLD_arr_matrix = coo_matrix(QueenslandFLD_arr)
# # QueenslandFLD_df_tfidf_3gram_matrix = coo_matrix(QueenslandFLD_df_tfidf_3gram)
# # QueenslandFLD_df_tfidf_3gram_hstack = hstack([QueenslandFLD_arr_matrix,QueenslandFLD_df_tfidf_3gram_matrix]).toarray()



# # save to npy files
# np.save('NepalEQ_df_tfidf_1gram_hstack.npy', NepalEQ_df_tfidf_1gram_hstack)
# np.save('NepalEQ_df_label.npy', NepalEQ_df.label)
# np.save('QueenslandFLD_df_tfidf_1gram_hstack.npy', QueenslandFLD_df_tfidf_1gram_hstack)
# np.save('QueenslandFLD_df_label.npy', Queensland_FLD.label)




#### Fasttext + hashtags + Retweets

In [ ]:
# # Create a sparse matrix in COO format NepalEQ_df_embeddings_fasttext
# NepalEQ_df_embeddings_fasttext_matrix = coo_matrix(NepalEQ_df_embeddings_fasttext)
# NepalEQ_df_embeddings_fasttext_hstack = hstack([NepalEQ_arr_matrix,NepalEQ_df_embeddings_fasttext_matrix]).toarray()

# # Create a sparse matrix in COO format QueenslandFLD_df_embeddings_fasttext
# QueenslandFLD_df_embeddings_fasttext_matrix = coo_matrix(QueenslandFLD_df_embeddings_fasttext)
# QueenslandFLD_df_embeddings_fasttext_hstack = hstack([QueenslandFLD_arr_matrix,QueenslandFLD_df_embeddings_fasttext_matrix]).toarray()

# np.save('NepalEQ_df_embeddings_fasttext_hstack.npy', NepalEQ_df_embeddings_fasttext_hstack)
# np.save('QueenslandFLD_df_embeddings_fasttext_hstack.npy', QueenslandFLD_df_embeddings_fasttext_hstack)

#### Glove + hashtags + Retweets

In [ ]:
# # Create a sparse matrix in COO format NepalEQ_df_embeddings_glove
# NepalEQ_df_embeddings_glove_matrix = coo_matrix(NepalEQ_df_embeddings_glove)
# NepalEQ_df_embeddings_glove_hstack = hstack([NepalEQ_arr_matrix,NepalEQ_df_embeddings_glove_matrix]).toarray()

# # Create a sparse matrix in COO format QueenslandFLD_df_embeddings_glove
# QueenslandFLD_df_embeddings_glove_matrix = coo_matrix(QueenslandFLD_df_embeddings_glove)
# QueenslandFLD_df_embeddings_glove_hstack = hstack([QueenslandFLD_arr_matrix,QueenslandFLD_df_embeddings_glove_matrix]).toarray()

# np.save('NepalEQ_df_embeddings_glove_hstack.npy', NepalEQ_df_embeddings_glove_hstack)
# np.save('QueenslandFLD_df_embeddings_glove_hstack.npy', QueenslandFLD_df_embeddings_glove_hstack)

#### word2vec + hashtags + Retweets

In [ ]:
# # Create a sparse matrix in COO format NepalEQ_df_embeddings_word2vec
# NepalEQ_df_embeddings_word2vec_matrix = coo_matrix(NepalEQ_df_embeddings_word2vec)
# NepalEQ_df_embeddings_word2vec_hstack = hstack([NepalEQ_arr_matrix,NepalEQ_df_embeddings_word2vec_matrix]).toarray()

# # Create a sparse matrix in COO format QueenslandFLD_df_embeddings_word2vec
# QueenslandFLD_df_embeddings_word2vec_matrix = coo_matrix(QueenslandFLD_df_embeddings_word2vec)
# QueenslandFLD_df_embeddings_word2vec_hstack = hstack([QueenslandFLD_arr_matrix,QueenslandFLD_df_embeddings_word2vec_matrix]).toarray()

# np.save('NepalEQ_df_embeddings_word2vec_hstack.npy', NepalEQ_df_embeddings_word2vec_hstack)
# np.save('QueenslandFLD_df_embeddings_word2vec_hstack.npy', QueenslandFLD_df_embeddings_word2vec_hstack)

#### bert + hashtags + Retweets

In [ ]:
# # Create a sparse matrix in COO format NepalEQ_df_bert_input
# NepalEQ_df_bert_input_matrix = coo_matrix(NepalEQ_df_bert_input)
# NepalEQ_df_bert_input_hstack = hstack([NepalEQ_arr_matrix,NepalEQ_df_bert_input_matrix]).toarray()

# # Create a sparse matrix in COO format QueenslandFLD_df_bert_input
# QueenslandFLD_df_bert_input_matrix = coo_matrix(QueenslandFLD_df_bert_input)
# QueenslandFLD_df_bert_input_hstack = hstack([QueenslandFLD_arr_matrix,QueenslandFLD_df_bert_input_matrix]).toarray()

#### bow + hashtags + Retweets

In [ ]:
# #NepalEQ_df_em_1gram
# # Create a sparse matrix in COO format
# NepalEQ_df_em_1gram_matrix = coo_matrix(NepalEQ_df_em_1gram)
# NepalEQ_df_em_1gram_hstack = hstack([NepalEQ_arr_matrix,NepalEQ_df_em_1gram_matrix]).toarray()

# # NepalEQ_df_em_2gram_matrix = coo_matrix(NepalEQ_df_em_2gram)
# # NepalEQ_df_em_2gram_hstack = hstack([NepalEQ_arr_matrix,NepalEQ_df_em_2gram_matrix]).toarray()

# #NepalEQ_df_em_3gram_matrix = coo_matrix(NepalEQ_df_em_3gram)
# #NepalEQ_df_em_3gram_hstack = hstack([NepalEQ_arr_matrix,NepalEQ_df_em_3gram_matrix]).toarray()


# # Create a sparse matrix in COO format
# QueenslandFLD_df_em_1gram_matrix = coo_matrix(QueenslandFLD_df_em_1gram)
# QueenslandFLD_df_em_1gram_hstack = hstack([QueenslandFLD_arr_matrix,QueenslandFLD_df_em_1gram_matrix]).toarray()

# # QueenslandFLD_df_em_2gram_matrix = coo_matrix(QueenslandFLD_df_em_2gram)
# # QueenslandFLD_df_em_2gram_hstack = hstack([QueenslandFLD_arr_matrix,QueenslandFLD_df_em_2gram_matrix]).toarray()

# # QueenslandFLD_df_em_3gram_matrix = coo_matrix(QueenslandFLD_df_em_3gram)
# # QueenslandFLD_df_em_3gram_hstack = hstack([QueenslandFLD_arr_matrix,QueenslandFLD_df_em_3gram_matrix]).toarray()

# np.save('NepalEQ_df_em_1gram_hstack.npy', NepalEQ_df_em_1gram_hstack)
# np.save('QueenslandFLD_df_em_1gram_hstack.npy', QueenslandFLD_df_em_1gram_hstack)

#### POS + hashtags + Retweets

In [ ]:
# # Convert the new DataFrame to a NumPy array for NepalEQ_df
# NepalEQ_POS_selected = NepalEQ_df[['Noun_Count_normalized', 'Verb_Count_normalized', 'Adjective_Count_normalized', 'Adverb_Count_normalized']]
# # Convert the dataframe to a 2D array
# NepalEQ_POS_arr = NepalEQ_POS_selected.to_numpy()

# NepalEQ_POS_matrix = coo_matrix(NepalEQ_POS_arr)
# NepalEQ_POS_hstack = hstack([NepalEQ_arr_matrix,NepalEQ_POS_matrix]).toarray()


# # Convert the new DataFrame to a NumPy array for QueenslandFLD_df
# QueenslandFLD_POS_selected = QueenslandFLD_df[['Noun_Count_normalized', 'Verb_Count_normalized', 'Adjective_Count_normalized', 'Adverb_Count_normalized']]
# # Convert the dataframe to a 2D array
# QueenslandFLD_df_POS_arr = QueenslandFLD_POS_selected.to_numpy()

# QueenslandFLD_POS_matrix = coo_matrix(QueenslandFLD_df_POS_arr)
# QueenslandFLD_POS_hstack = hstack([QueenslandFLD_arr_matrix,QueenslandFLD_POS_matrix]).toarray()


# np.save('NepalEQ_POS_hstack.npy', NepalEQ_POS_hstack)
# np.save('QueenslandFLD_POS_hstack.npy', QueenslandFLD_POS_hstack)